In [5]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
from sklearn.cluster import DBSCAN, KMeans
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import threading as trd
import pywt 


import warnings
warnings.filterwarnings('ignore')

In [2]:
y, sr = librosa.load('../DataSets/training-b/b0015.wav')

In [3]:
def statisticFeature(X):
    mean=np.mean(X)
    std = np.std(X)
    maxamp = np.max(X)
    minamp = np.min(X)
    var = np.var(X)
    median = np.median(X)
    average = np.average(X)
    return mean,std,maxamp,minamp,var,median,average

In [4]:
def extract_feature(y,sr):
    #liberosa feature extraction base on signal processes
    stftt = statisticFeature(np.abs(librosa.stft(y)))
    mfccs = statisticFeature(librosa.feature.mfcc(y, sr))
    chromaStft= statisticFeature(librosa.feature.chroma_stft(y,sr))
    chromaCqt = statisticFeature(librosa.feature.chroma_cqt(y, sr))
    chromaCens = statisticFeature(librosa.feature.chroma_cens(y, sr))
    melspectrogram = statisticFeature(librosa.feature.melspectrogram(y, sr))
    mfcc = statisticFeature(librosa.feature.mfcc(y, sr, n_mfcc=10, dct_type=2, norm='ortho'))
    rmse = statisticFeature(librosa.feature.rmse(y))
    spectral_centroid = statisticFeature(librosa.feature.spectral_centroid(y,sr))
    spectral_bandwidth = statisticFeature(librosa.feature.spectral_bandwidth(y,sr))
    spectral_contrast = statisticFeature(librosa.feature.spectral_contrast(y,sr, n_fft=2048, hop_length=512, freq=None, fmin=200.0, n_bands=6, quantile=0.02, linear=False))
    spectral_flatness = statisticFeature(librosa.feature.spectral_flatness(y))
    spectral_rolloff = statisticFeature(librosa.feature.spectral_rolloff(y,sr))
    poly_features = statisticFeature(librosa.feature.poly_features(y,sr))
    tonnetz = statisticFeature(librosa.feature.tonnetz(y,sr))
    zero_crossing_rate = statisticFeature(librosa.feature.zero_crossing_rate(y))
    #wavelet feature extraction
    (DWT2D_A,DWT2D_D) = pywt.dwt(y, 'haar')
    DWT2D_A = statisticFeature(DWT2D_A)
    DWT2D_D = statisticFeature(DWT2D_D)
    
    wavedec = pywt.wavedec(y, 'db1', level=2)
    cA2, cD2, cD1 = wavedec
    wavedecA = statisticFeature(cA2)
    wavedecCD2 = statisticFeature(cD2)
    wavedecCD1 = statisticFeature(cD1)
    

    return stftt,mfccs,chromaStft,chromaCqt,chromaCens,melspectrogram,mfcc,rmse,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,spectral_rolloff,poly_features,tonnetz,zero_crossing_rate,DWT2D_A,DWT2D_D,wavedecA,wavedecCD2,wavedecCD1

In [43]:
staticFeature = ['mean','std','max','min','var','median','average']
listFeature = ['stftt','mfccs','chromaStft','chromaCqt','chromaCens','melspectrogram','mfcc','rmse','spectral_centroid','spectral_bandwidth','spectral_contrast','spectral_flatness','spectral_rolloff','poly_features','tonnetz','zero_crossing_rate','DWT2D_A','DWT2D_D','wavedecA','wavedecCD2','wavedecCD1']
z = []
for a in listFeature:
    for b in staticFeature:
        z.append(a+'_'+b)
        
featureNames = np.array(z)
featureNames.shape

(112,)

In [52]:
temp = extract_feature(y,sr)
temp = np.array(temp)
temp = temp.reshape(1,112)
temp = pd.DataFrame(temp,columns=featureNames)


In [53]:
temp

,stftt_mean,stftt_std,stftt_max,stftt_min,stftt_var,stftt_median,stftt_average,mfccs_mean,mfccs_std,mfccs_max,...,tonnetz_var,tonnetz_median,tonnetz_average,zero_crossing_rate_mean,zero_crossing_rate_std,zero_crossing_rate_max,zero_crossing_rate_min,zero_crossing_rate_var,zero_crossing_rate_median,zero_crossing_rate_average
0,0.325092,6.094154,440.248749,2.127059e-09,37.138714,0.000001,0.325092,-2.095917,94.856278,174.877367,...,0.006366,0.024693,0.019109,0.001078,0.000685,0.00293,0.0,4.692174e-07,0.000977,0.001078


In [31]:
temp2 = pd.DataFrame(temp)

In [32]:
temp2

,0,1,2,3,4,5,6
0,3.250917e-01,6.094154e+00,440.248749,2.127059e-09,3.713871e+01,1.096093e-06,3.250917e-01
1,-2.095917e+00,9.485628e+01,174.877367,-4.326455e+02,8.997713e+03,1.278297e+01,-2.095917e+00
2,8.272410e-01,1.469781e-01,1.000000,1.278598e-01,2.160256e-02,8.564955e-01,8.272410e-01
3,4.785376e-01,2.820080e-01,1.000000,1.431388e-02,7.952849e-02,4.283214e-01,4.785376e-01
4,2.418785e-01,1.575694e-01,0.609219,0.000000e+00,2.482813e-02,2.481089e-01,2.418785e-01
5,4.359626e+00,1.026073e+02,10139.973407,2.015827e-15,1.052827e+04,1.617058e-12,4.359626e+00
6,-1.382151e+01,1.330467e+02,174.877367,-4.326455e+02,1.770143e+04,2.341593e+01,-1.382151e+01
7,1.748513e-01,9.078682e-02,0.593323,4.220367e-02,8.242247e-03,1.588816e-01,1.748513e-01
8,3.342113e+01,2.316313e+01,328.723634,1.079709e+01,5.365307e+02,2.862469e+01,3.342113e+01
9,7.896270e+01,6.165616e+01,948.289859,3.540023e+01,3.801483e+03,6.925137e+01,7.896270e+01


In [27]:
temp3 = temp.reshape(112)

In [33]:
temp4 = pd.DataFrame(temp3)
temp4

,0
0,3.250917e-01
1,6.094154e+00
2,4.402487e+02
3,2.127059e-09
4,3.713871e+01
5,1.096093e-06
6,3.250917e-01
7,-2.095917e+00
8,9.485628e+01
9,1.748774e+02


In [20]:
(DWT2D_A,DWT2D_D) = pywt.dwt(y, 'haar')
DWT2D_A = statisticFeature(DWT2D_A)
DWT2D_D = statisticFeature(DWT2D_D)

In [23]:
wavedec = pywt.wavedec(y, 'db1', level=2)
cA2, cD2, cD1 = wavedec
wavedecA = statisticFeature(cA2)
wavedecCD2 = statisticFeature(cD2)
wavedecCD1 = statisticFeature(cD1)

In [24]:
wavedecA

(-0.12853818,
 0.3727784,
 1.8261073,
 -1.9998453,
 0.13896371,
 -0.118192405,
 -0.12853818)